image data

In [1]:
import os
import json
import datetime

import pandas as pd
import numpy as np

import driverlessai
driverlessai.__version__

'1.10.1.3'

In [4]:
# Driverless AIのuser nameとpasswordの読み込み
with open(os.path.join('../../..', 'idpass.json')) as f:
    idpass = json.load(f)

In [5]:
def get_dai_client(daiaddress, daipassword) -> 'driverlessai._core.Client':
    '''
    DAIサーバへの接続
    ----------
    daiaddress : str
    daipassword : str
    '''
    print('----- start server connection : get_dai_client -----')
    # Driverless AIサーバーへの接続
    dai = driverlessai.Client(address=daiaddress, username=idpass['id'], password=daipassword)
    return dai

def get_dataset(daiobj, dataname, dataurl) -> 'driverlessai._datasets.Dataset': 
    '''
    データオブジェクトの取得
    ----------
    daiobj : driverlessai._core.Client
    dataname : str
    dataurl : str
    '''
    print('----- start get data : get_dataset -----')
    # DAI上のデータ一覧
    uploaded_data = {i.name:i.key for i in daiobj.datasets.list()}
    print('Uploaded data name : key >> ', uploaded_data)

    # データ取得
    if dataname in uploaded_data.keys():
        print('Data is already uploaded in DAI')
        ds = daiobj.datasets.get(uploaded_data[dataname]) 
    else:
        print('Data is uploading to DAI.')
        ds = daiobj.datasets.create(data=dataurl, data_source='s3')
    
    return ds

def get_experiment(daiobj, dataobj, target_column, task, drop_columns, enable_gpus) -> 'driverlessai._experiments.Experiment':
    '''
    Experimentの実行とExperimentオブジェクトの取得
    ----------
    daiobj : driverlessai._core.Client
    dataobj : driverlessai._datasets.Dataset
    target_column : str
    task : str
    drop_columns : List[str]
    enable_gpus : bool
    '''
    print('----- start experiment : get_experiment -----')
    # Experiment設定
    dai_settings = {
        'train_dataset': dataobj, 
        'target_column': target_column,
        'task': task,
        'drop_columns': drop_columns,
        'enable_gpus': enable_gpus
    }
    # Experimentの実行
    ex = daiobj.experiments.create(**dai_settings)
    return ex

In [11]:
df_expperiments_info = pd.read_csv('../experiment_meta_data/Experiments_Params_0315_Gpu.csv')
df_expperiments_info

,try_n,data_name,s3url,target_column,task,drop_columns,enable_gpus
0,3,car_deals.zip,s3://h2o-public-test-data/bigdata/server/Image...,Price,regression,"Manufacturer,Model,Year,Category,Mileage,FuelT...",True
1,3,histopathology_train.zip,s3://h2o-public-test-data/bigdata/server/Image...,label,classification,NaN,True
